In [3]:
import pandas as pd

df = pd.read_csv('../../assets/doc/OutdoorClothingCatalog_1000.csv')
df.head(5)

,Unnamed: 0,name,description
0,0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...


In [11]:
from langchain.chains.retrieval_qa.base import RetrievalQA #检索QA链，在文档上进行检索
from langchain_community.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch #向量存储

In [7]:
file = '../../assets/doc/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()
docs[0]

Document(metadata={'source': '../../assets/doc/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [8]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

C:\Users\62552\AppData\Local\Temp\ipykernel_28556\2008307582.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model='text-embedding-3-small')


In [9]:
embed = embeddings.embed_query("Hi my name is Harrison")
embed[:5], len(embed)

([0.005679303826556005,
  -0.005919360118681657,
  -0.06266986116010925,
  0.022632133407379053,
  -0.051341637903750785],
 1536)

In [12]:
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

d:\software\anaconda\envs\transformer-learn\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [14]:
query = "Please suggest a shirt with sunblocking"

docs = db.similarity_search(query)
docs[0]

Document(metadata={'source': '../../assets/doc/OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [15]:
retriever = db.as_retriever()

In [17]:
llm = ChatOpenAI(model_name="gpt-4o") 

C:\Users\62552\AppData\Local\Temp\ipykernel_28556\2615805828.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-4o")


In [16]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [20]:
resp = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in text")
resp

"Here is a list of shirts with sun protection:\n\n1. **Sun Shield Shirt**\n   - Description: High-performance sun shirt offering UPF 50+ protection, blocking 98% of harmful UV rays. Made with 78% nylon and 22% Lycra Xtra Life fiber. Slightly fitted and falls at hip. Features moisture-wicking, quick-drying, and abrasion resistance.\n   - Care: Handwash, line dry.\n\n2. **Men's Tropical Plaid Short-Sleeve Shirt**\n   - Description: Lightest hot-weather shirt with UPF 50+ rating. Made of 100% polyester, it is wrinkle-resistant with front and back cape venting and two front bellows pockets.\n   - Fit: Traditional fit, relaxed through chest, sleeve, and waist.\n   - Care: Imported.\n\n3. **Men's Plaid Tropic Shirt, Short-Sleeve**\n   - Description: Ultracomfortable shirt with UPF 50+ protection. Originally for fishing, it offers great coverage for travel. SunSmart technology blocks 98% of UV rays. Made with 52% polyester and 48% nylon, it is wrinkle-free and moisture-wicking.\n   - Features